# HW1 - Intro to the Map Reduce Paradigm  
__ `MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Spring 2018`__

Welcome to Machine Learning at Scale! This first homework assignment introduces one of the core strategies in distributed processing: divide and conquer. We'll use the simplest of tasks, word counting, to illustrate the difference between a scalable and non-scalable algorithm. You will be working with the text of _Alice in Wonderland_ to put these ideas into practice using Python and Bash scripting. By the end of this week you should be able to:
* ... __describe__ the Bias-Variance tradeoff as it applies to Machine Learning.
* ... __explain__ why we consider word counting to be an "Embarrassingly Parallel" task.
* ... __estimate__ the runtime of embarrassingly parallel tasks using "back of the envelope" calculations.
* ... __implement__ a Map Reduce algorithm using the Command Line.
* ... __set-up__ a Docker container and know why we use them for this course.

You will also  become familiar (if you aren't already) with `defaultdict`, `re` and `time` in Python, linux piping and sorting, and Jupyter magic commands `%%writefile` and `%%timeit`. __Please refer to the `README` for detailed submission instructions__.

__IMPORTANT:__ If you're not familiar with linux, you should read the following tutorial reagrding **piping** and **redirecting**: https://ryanstutorials.net/linuxtutorial/piping.php You will need to understand the differences to answer some of the later questions.


# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [1]:
# confirm you are running Python 3
import sys
sys.version_info

sys.version_info(major=3, minor=6, micro=1, releaselevel='final', serial=0)

In [2]:
!python3 --version
#! apt-get install bc
#! apt-get install man

Python 3.6.1 :: Anaconda 4.4.0 (64-bit)


In [3]:
# imports
import re
import sys

Create a folder for any data you download locally.

In [4]:
!mkdir data
# NOTE: the contents of this directory will be ignored by git.

mkdir: cannot create directory 'data': File exists


# Question 1: Introductions

`The Caterpillar and Alice looked at each other for some time in silence: at last the Caterpillar took the hookah out of its mouth, and addressed her in a languid, sleepy voice. "Who are you?" said the Caterpillar.`   
<div style="text-align: right"> -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4 </div>


__Tell us about yourself! Briefly describe where you live, how far along you are in MIDS, what other classes you are taking and what you want to get out of w261.__

Hi, my name is Chandra Shekar Bikkanur. I am from Dallas, Texas, USA. This is my 5th semester in MIDS. So far, I have completed w201, w203, w207, w266, w271 and w205 courses. This semester, I am only taking w261 course. I want to be familiar with the Machine Learning implementations/solutions for large(petabyte/exabyte) datasets. From 261, I want to learn how parallel processing can be utilized to process large datasets in the context of Machine Learning.

# Question 2: Bias - Variance
__In 1-2 sentences explain the bias-variance trade off. Describe what it means to "decompose" sources of error. How is this used in machine learning?__ Please use mathematical equation(s) to support your explanation. (Use `$` signs to take advantage of $\LaTeX$ formatting, eg. `$f(x)$` will look like: $f(x)$). Please also cite any sources that informed your answer.

* **Bias:** Bias of a model is the error due to the average difference between expected prediction to the ground truth value.
* **Variance:** Variance of a model is the error due to the variablity in the predicted value accross different models.
* **Bias-Variance trade off:** As bias of a model is increased, that will inturn decrease the variance of the model and vice versa. We need to find the optimized bias and variance values which will lower the prediction errors to as low as irreducible error.  
* **Decompose source of error:** The source of error for any predicted model is a combination of squared bias, variance and irreducible error. So, decomposing the source of error means to find out the individual errors due to bias, variance and irreducible error. If we have to predict $Y$ using $X$ with a linear regression model, it can be expressed as $Y = f(X) + \epsilon$, where $\epsilon$ is normally distributed irreducible error. If the predicted model is $f\hat(X)$, then the prediction error can be decomposed as below:
$$
Err(x) = E[(Y-f\hat(x))^2]
$$
$$
Err(x) = (E[f\hat(x)]-f(X))^2 + E[(f\hat(x)-E[f\hat(x)])^2] + \sigma_{e}^2
$$
$$
Err(x) = \text{Bias}^2 + \text{Varaiance} + \text{Irreducible Error}
$$
* **Usage in Machine Learning :** In machine learning, we use the decomposition of source of error to see the variance and bias values. This is needed to figure out if the model is overfitting or underfitting the ground truth data. Decomposition of error is also the basis for regression regularization methods such as L1 (Lasso Regression) and L2 (Ridge Regression)  

_Sources:_ http://scott.fortmann-roe.com/docs/BiasVariance.html, https://theclevermachine.wordpress.com/2013/04/21/model-selection-underfitting-overfitting-and-the-bias-variance-tradeoff/


# Question 3: Tokenizing
A number of our assignments this term will involve extracting information from text. A common preprocessing step when working with raw files is to 'tokenize' (i.e. extract words from) the text. Within the field of Natural Language Processing a lot of thought goes into what specific tokenizing makes most sense for a given task. For example, you might choose to remove punctuation or to consider punctuation symbols  'tokens' in their own right. __In this question you'll use the Python `re` module to create a tokenizer to use when you perform WordCount on the _Alice In Wonderland_ text.__

### Q3 Tasks:

* __a) short response:__ In the Naive Bayes algorithm (which we'll implement next week), we'll estimate the _likelihood_ of a word by counting the number of times it appears and dividing by the size of the vocabulary (total number of unique words). Using the text: *"Alice had an adventure that took alice to wonderland"*, give a concrete example of how two different tokenizers could cause us to get two different results on this calculation. [`HINT`: _you should not need to read up on Naive Bayes to answer this question_]  
  

* __b) short response:__ When tokenizing in this assignment we'll remove punctuation and discard numerical digits by making everything lowercase and then capturing only consecutive letters a to z. Suppose __`tokenize(x)`__ is a Python function that performs the desired tokenization. What would __`tokenize("By-the-bye, what became of Alice's 12 hats?!")`__ output? Type the answer in the space provided below.


* __c) code:__  Fill in the regular expression pattern in the cell labeled `part c` so that the subsequent call to `re.findall(RE_PATTERN, ...)` returns the tokenization described above. [`HINT`: _we've taken care of the lowercase part for you. If regex is new to you, check out the [`re`  documentation](https://docs.python.org/3/library/re.html) or [this PyMOTW tutorial](https://pymotw.com/2/re/)._]

### Q3 Student Answers:
> __a)__ 
* If a tokenizer treats every word as a token, then **Alice** and **alice** will be treated as two different words and the corresponding likelihood of the word would be differnt.
* If a tokenizer converts every word to its lower case, then **Alice** will be converted to **alice** and we will have 2 occurences of word **alice** thereby changing the likelihood of the word
* If a tokenizer converts every word to its lowercase **and** removes stop words such as 'a', 'the', 'of', etc., then the likelihood of the word would vary as the unique words would be reduced.


> __b)__ by the bye what became of alice s hats

In [5]:
# Part C - Fill in the regular expression
RE_PATTERN = re.compile("[!.?\-',0-9;:~`@#$%^&*()_+=<>]+")       

In [6]:
# Tokenize - DO NOT MODIFY THIS CELL, just run it as is to check your pattern
words = re.findall(RE_PATTERN, "By-the-bye, what became of Alice's 12 hats?!".lower())
print(words)

['-', '-', ',', "'", '12', '?!']


# Load the Data
` "Please would you tell me", said Alice, a little timidly, for she was not quite sure whether it was good manners for her to speak first, "why your cat grins like that?"`  
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4  </div>

For the main part of this assignment we'll be working with the free plain text version of _Alice's Adventures in Wonderland_ available from Project Gutenberg. __Use the first two cells below to download this text and preview the first few lines.__ 

In [7]:
# Download Full text 
# NOTE: feel free to replace 'curl' with 'wget' or equivalent command of your choice.
!curl "http://www.gutenberg.org/files/11/11-0.txt" -o data/alice.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  169k  100  169k    0     0   779k      0 --:--:-- --:--:-- --:--:--  991k


In [8]:
# Take a peak at the first few lines
!head -n 6 data/alice.txt
# NOTE: If you are working in JupyterLab on Docker you may not see the output 
# below due to an encoding issue... in that case, use a terminal on Docker to 
# execute this head command and confirm that the file has downloaded properly, 
# this encoding issue should not affect your work on subsequent HW items.

We'd also like you to develop a habit of creating small files with simulated data for use in developing, debugging and testing your code. The jupyter magic command `%%writefile` is a convenient way to do this. __Run the following cells to create a test data file for use in our word counting task.__

In [9]:
%%writefile data/alice_test.txt
This is a small test file. This file is for a test.
This small test file has two small lines.

Overwriting data/alice_test.txt


In [10]:
# confirm the file was created in the data directory using a grep command:
!ls data | grep test

alice_test.txt
alice_test.txt.output


# Question 4: Word Count in Python

Over the course of this term you will also become very familiar writing Python programs that read from standard input and using Linux piping commands to run these programs and save their output to file. __In this question you will write a short python script to perform the Word Count task and then run your script on the _Alice in Wonderland_ text__. You can think of this like a 'baseline implementation' that we'll later compare to the parallelized version of the same task.

### Q4 Tasks:

* __a) code:__ Complete the Python script in the file __`wordCount.py`__. Read the docstrings carefully to be sure you understand the expected behavior of this function. Please do not code outside of the marked location.


* __b) testing:__ Run the cell marked `part b` to call your script on the test file we created above. Confirm that your script returns the correct counts for each word by visually comparing the output to the test file. 


* __c) results:__ When you are confident in your implementation, run the cell marked `part c` to count the number of occurrences of each word in _Alice's Adventures in Wonderland_. In the same cell we'll pipe the output to file. Then use the provided `grep` commands to check your answers.


* __d) short response:__ Suppose you decide that you'd really like  a word and its plural (e.g. 'hatter' and 'hatters' or 'person' and 'people') to be counted as the same word. After we have run the wordcount would it be more efficient to post-process your output file or discard your output file and start the analysis over with a new tokenizer? Explain your reasoning briefly. 

### Q4 Student Answers:
> __a-c)__ _Complete the coding portions of this question before answering 'e'._

> __d)__ It would be more efficient to post-process the file to combine plural and singular words to generate the wordcount. As majority of the words' plural form consists of same starting letters (apple -> apples), we could search for the existence of the plural form of the word and add the count as desired. On the other hand, if we have to use a new tokenizer and preprocess the entire corpora, the complexity will be huge based on the corpora size. 

In [11]:
# part a - DO YOUR WORK IN wordCount.py

In [12]:
# part b - DO NOT MODIFY THIS CELL, just run it as is to test your script
!python wordCount.py < data/alice_test.txt

this 	 3
is 	 2
a 	 2
small 	 3
test 	 3
file 	 3
for 	 1
has 	 1
two 	 1
lines 	 1


In [13]:
# part c - DO NOT MODIFY THIS CELL, just run it as is to perform the word count.
!python wordCount.py < data/alice.txt > data/alice_counts.txt

Take a look at the first 10 words & their counts.

In [14]:
!head data/alice_counts.txt

project 	 87
gutenberg 	 93
s 	 219
alice 	 403
adventures 	 12
in 	 431
wonderland 	 8
by 	 78
lewis 	 4
carroll 	 4


__Check your results:__ How many times does the word "alice" appear in the book? 

In [15]:
# EXPECTED OUTPUT: 403
!grep alice data/alice_counts.txt

alice 	 403


__Check your results:__ How many times does the word "hatter" appear in the book? 

In [16]:
# EXPECTED OUTPUT: 56
!grep hatter data/alice_counts.txt

hatter 	 56
hatters 	 1


__Check your results:__ How many times does the word "queen" appear in the book? 

In [17]:
# EXPECTED OUTPUT: 75
!grep queen data/alice_counts.txt

queen 	 75
queens 	 1


# Question 5: Unix Sorting
Another common task in this course's assignments will be to make strategic use of sorting. To illustrate what we mean by 'strategic,' let's return to the (admitedly artificial) scenario posed in Question 4d): _we have a file of word counts but we want to find all pairs of words and plurals (eg. 'queen' and 'queens') to join their counts into a single record_. Suppose we were going to do this by hand. A logical approach would be to start with the first word and read through each subsequent word to check if they are a singular-plural pair, then repeat this process for each word in the file.    

__IMPORTANT NOTE__: For this question, pretend that all plurals simply end with 's'.

### Q5 Tasks:
* __a) short response + code:__ In the worst case scenario, how many comparisons will we have to make to perform the process described above? Use $\LaTeX$ formatting to show the calculations that led to your answer. What is the Big O complexity of this "algorithm"? [`HINTs:` _To answer the first part of this question you'll want to know how many words are in your_ `alice_counts.txt` _file -- in the space provided below, write a unix command to check. If you need a Big O notation refresher, here's_ a [blog post](https://rob-bell.net/2009/06/a-beginners-guide-to-big-o-notation/), a [cheatsheet](http://bigocheatsheet.com), and a [thorough explanation](http://pages.cs.wisc.edu/~vernon/cs367/notes/3.COMPLEXITY.html).]


* __b) short response:__ If the word count file were sorted alphabetically, how would your approach to this task change? How many comparisons would you have to make? What would the Big O complexity of this new strategy be? *(Remember that for this question, we're assuming all plural words simply end with 's')*


* __c) short response:__ What is the Big O complexity of the fastest sorting algorithms? When we take the time to sort into account, does it make sense to alphabetize before combining singular-plural pairs? [`HINT:` *When it comes to sorting algorithms -- Wikipedia is your friend, as is the cheatsheet from part a.*]


* __d) code:__ Write a unix command to sort your word count file alphabetically. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) the results to `data/alice_counts_A-Z.txt`. [`HINT:` if Unix sort commands are new to you, start with [this biowize blogpost](https://biowize.wordpress.com/2015/03/13/unix-sort-sorting-with-both-numeric-and-non-numeric-keys/) or [this unixschool tutorial](http://www.theunixschool.com/2012/08/linux-sort-command-examples.html)]


* __e) code:__ A (slightly) more likely scenario is that we'd want to look at the top 10 most frequent words that appear in the book. Write a unix command to sort your word count file from highest to lowest count. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) your results to `data/alice_counts_sorted.txt`; then run the provided cell to print the top ten words. Compare your output to the expected output we provide.

### Q5 Student Answers:
> __a)__ In our wordcount file, there are 3009 words in total. If we have to compare each word with every other word to see if it is a plural form, we need to compare words in a **nested for loop** pattern; also, we could exclude comparing the word with itself which will be 3009 less comparisons.
$$
\text{Total number of comparisions for 3009 words(excluding itself): } (3009*3009)-3009 = 9051072 \text{ Comparisons}
$$
$$
\text{The Big O complexity for this comparison/algoritm is: O(n^2)}
$$

> __b)__ If we were given an alphabetically sorted wordcount file, we could do a binary search for the plural form of a given word. The complexity for one word binary-search would be $O(log(n))$; we need perform this for all words which will make the overall complexity $O(n*log(n))$

> __c)__ Big O complexity of the fastest sorting algorithm is $O(n*log(n))$. If we do not sort the wordcount and perform combining singular-plural pairs, it will take $O(n^2)$ complexity; whereas if the wordcount file is sorted($O(n*log(n))$), it will take additional $O(n*log(n)$ comparisons to combine the pairs. It does not make sense to alphabetize the words, if all we want to do is combine the singular-plural pairs.


In [18]:
# part a - write a unix command to check how many records are in your word count file
!wc -l data/alice_counts.txt # counting the number of lines in the file to check the word_count

3009 data/alice_counts.txt


In [19]:
# part d - unix command to sort your word counts alphabetically 
!sort data/alice_counts.txt> data/alice_counts_A-Z.txt 

In [20]:
# part d - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_A-Z.txt   

a 	 690
abide 	 2
able 	 1
about 	 102
above 	 3
absence 	 1
absurd 	 2
accept 	 1
acceptance 	 1
accepted 	 2


In [21]:
# part e - unix command to sort your word counts from highest to lowest count
!sort -t "$(echo '\t')" -k2nr,2 data/alice_counts.txt > data/alice_counts_sorted.txt 

In [22]:
# part e - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_sorted.txt  

the 	 1818
and 	 940
to 	 809
a 	 690
of 	 631
it 	 610
she 	 553
i 	 545
you 	 481
said 	 462


<table>
<th>expected output for (d):</th>
<th>expected output for (e):</th>
<tr><td><pre>
a          690
abide      2
able       1
about      102
above      3
absence    1
absurd     2
accept     1
acceptance 1
accepted   2
</pre></td>
<td><pre>
the	 1818
and	 940
to	 809
a	 690
of	 631
it	 610
she	 553
i	 545
you	 481
said	 462
</pre></td></tr>
</table>

# Question 6: Parallel Word Count (part 1)
What would happen if we tried to run our script on a much larger dataset? For one thing, it would take longer to run. However there is a second concern. The Python object that aggregates our counts (`defaultdict`) exists in memory on the machine running this notebook. If the vocabulary is too large for the memory space available we would crash the notebook. The solution? Divide and Conquer! Instead of running the script on the whole dataset at once, we could split our text up in to smaller 'chunks' and process them independently of each other. __In this question you'll use a bash script to "parallelize" your Word Count.__


### Q6 Tasks:
* __a) read provided code:__ The bash script `pWordCount_v1.sh` takes an input file, splits it into a specified number of 'chunks', and then applies an executable of your choice to each chunk. Read through this code and make sure you understand each line before you proceed. [`HINT:` _For now, ignore the 'student code' section -- you'll use that in part c._]


* __b) short response:__ Below we've provided the command to use this script to apply your analysis (`wordCount.py`) to the _Alice_ text in 4 parallel processes. We'll redirect the results into a file called `alice_pCounts.txt.` Run this analysis and compare the count for the word 'alice' to your answer from Question 4. Explain what went wrong and describe what we have to add to `pWordCount_v1.sh` to fix the problem.


* __c) code:__ We've provided a python script, `aggregateCounts_v1.py`, which reads word counts from standard input and combines any duplicates it encounters. Read through this script to be sure you understand how it is written. Then follow the instructions in `pWordCount_v1.sh` to make a one-line modification so that it accepts `aggregateCounts_v1.py` as a 4th argument and uses this script to combine the chunk-ed word counts. Run the cell below to confirm that you now get the correct results for your 'alice' count.

### Q6 Student Answers:
> __b)__ As we have decared $4$ parallel processes as an argument(=$m$) for pWordCount_v1.sh, the script divided the lines into 4 chunks and performed the wordcount.py on each of the 4 chunks. As the results from these parallel processes are **not reduced**, we have 4 occurances of _alice_ with the total counts equal to 403 (as we got from Q4)
* **Fix:** we need to provide the reducer code as an argument to pWordCount_v1.sh; and also in pWordCount_v1.sh script, we need to reduce the output received from the mapper.

In [23]:
# part b - make sure your scripts are executable (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v1.sh
!chmod a+x wordCount.py

In [24]:
# part b - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' 'wordCount.py' > 'data/alice_pCounts.txt'

In [25]:
# part b - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice 	 111
alice 	 127
alice 	 122
alice 	 43


In [26]:
# part c - make sure the aggregateCounts script is executable (RUN THIS CELL AS IS)
!chmod a+x aggregateCounts_v1.py

In [27]:
# part c - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v1.py' \
                   > 'data/alice_pCounts.txt'

In [28]:
# part c - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	403


In [29]:
!man sort

SORT(1)                          User Commands                         SORT(1)

NNAAMMEE
       sort - sort lines of text files

SSYYNNOOPPSSIISS
       ssoorrtt [_O_P_T_I_O_N]... [_F_I_L_E]...
       ssoorrtt [_O_P_T_I_O_N]... _-_-_f_i_l_e_s_0_-_f_r_o_m_=_F

DDEESSCCRRIIPPTTIIOONN
       Write sorted concatenation of all FILE(s) to standard output.

       With no FILE, or when FILE is -, read standard input.

       Mandatory  arguments  to  long  options are mandatory for short options
       too.  Ordering options:

       --bb, ----iiggnnoorree--lleeaaddiinngg--bbllaannkkss
              ignore leading blanks

       --dd, ----ddiiccttiioonnaarryy--oorrddeerr
              consider only blanks and alphanumeric characters

       --ff, ----iiggnnoorree--ccaassee
              fold lower case to upper case characters

       --gg, ----ggeenneerraall--

# Question 7: Parallel Word Count (part 2)
Congratulations, you've just implemented a Map-Reduce algorithm! From here on out, we'll refer to the two python scripts you passed to `pWordCount_v1.sh` as '_mapper_' and '_reducer_'. The bash script itself served as our '_framework_' -- it split up the original input, then ___mapped___ our word counting script on to each chunk, then ___aggregated (a.k.a. reduced)___ the resulting files by piping them into our collation script.  Unfortunately, as you may have realized already, there is a major scalability concern with this particular implementation. __In this question you'll fix our implementation of parallel word count so that it will be scalable.__

__HINT:__ MapReduce uses the Merge-Sort algorithm under the hood. Linux `sort` command has a merge option which you can use to simiulate the MapReduce framework. Use the `man sort` command to find more information on this option. 

### Q7 Tasks:

* __a) short response:__ What is the potential scalability problem with the provided implementation of `aggregateCounts_v1.py`? Explain why this supposedly 'parallelized' Word Count wouldn't work on a really large input corpus.  [`HINT:` _see the intro to Q6_]


* __b) code:__ Fortunately, a 'strategic sort' can solve this problem. Read the instructions at the top of `pWordCount_v2.sh` carefully then make your changes that alphabetically sort the output from the mappers (`countfiles`) before piping them into the reducer script.


* __c) code:__ Write the main part of `aggregateCounts_v2.py` so that it takes advantage of the sorted input to add duplicate counts without storing the whole vocabulary in memory. Refer to the file docstring for more detailed instructions. Confirm that your implementation works by running it on both the test and true data files.


* __d) short response:__ If you are paying close attention, this rewritten reducer sets us up for a truly scalable solution, but doesn't get us all the way there. In particular, while we chunked our data so it can be processed by multiple mappers, we're still streaming the entire dataset through one reduce script. If the vocabulary is too large to fit on a single computer, we might split the word counts in half after sorting them, then perform the reducing on two separate machines. Explain what could go wrong with this approach. (For now, ignore the question of how we'd sort a dataset that is too large to fit on a single machine and just focus on what might be wrong about the result of this split-in-half reducing).


* __e) short response:__ Can you come up with a different way of splitting up the data that would allow us to perform the reducing on separate machines without needing any postprocessing? This is a theoretical question -- don't worry if you don't know how to implement your idea in a bash script, just describe how you'd want to split the sorted counts into different files to be reduced separately.

### Q7 Student Answers:

> __a)__ aggregateCounts_v1.py utilizes defaultdict dictionary for counting the words. If the vocabulary is too large, the dictionary which will be stored in-memory may cause the code to crash.

> __b-c)__ _complete the coding portions of this question before answering d and e._

> __d)__ If we split the sorted wordcounts data in half and perform `reducer` operation on 2 separate nodes, then there could be a chance where one word is split into 2 nodes for reducing($3$ `alice` to machine1 & 1 `alice` to machine$2$); and thus creating a duplicate of the word after reducing on 2 machines.

> __e)__ We need to split the data so that no word should go to 2 machines. Instead of looking for exact half split, we should split the data where occurances of one word is completely done around the half split. 

In [30]:
!chmod a+x pWordCount_v2.sh
!chmod a+x aggregateCounts_v2.py

In [31]:
# part c - test your code on the test file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice_test.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py'

a	2
file	3
for	1
has	1
is	2
lines	1
small	3
test	3
this	3


In [32]:
# part c - run your code on the Alice file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py' \
                   > 'data/alice_pCounts.txt'

In [33]:
# part c - confirm that your 'alice' count is correct (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	403


# Question 8: Scalability Considerations
In your reading for Week 1's live session, [Chapter1, section 2](https://lintool.github.io/MapReduceAlgorithms/MapReduce-book-final.pdf) of _Data Intensive Text Processing with MapReduce_, Lin and Dyer discuss a number of "Big Ideas" that underlie large scale processing: __scale "out," not "up"; assume failures are common; move processing to the data; process data sequentially and avoid random access; hide system-level details from the application developer; and seamless scalability.__ Part of our work this semester will be to interact with these ideas in a practical way, not just a conceptual one.

### Q8 Tasks:
* __a) short response:__ What do Lin and Dyer consider the two features of an "ideal algorithm" from a scalability perspective?


* __b) short response:__ The mapper script below (created on the fly using a little Jupyter magic) will help us illustrate the concept of scalability. Run the provided code which passes this mapper script to our parallel computation 'framework' and runs the 'analysis' on the _Alice_ text file. Note that we've omitted a reducer for simplicity. What do you observe about the time it takes to run this "algorithm" when we use 1, 2 and 4 partitions? Does it meet Lin and Dyer's criteria?


* __c) short response:__ Let's try something similar with your Word Count analysis. Run the provided code to time your implementation with 1, 2, 4 and 8 partitions. What do you observe about the runtimes? Does this match your expectation? Speculate about why we might be seeing these times. What conclusions should we draw about the scalability of our implementation? [`HINT:` _consider the limitations of both your machine and our implementation... there are some competing forces at work, what are they?_]


* __d) OPTIONAL:__ Which components of your Map-Reduce algorithm are affected by a change in the number of partitions? Does increasing the number of partitions increase or decrease the total time spent on each of these portions of the task? What tradeoff does this cause?

### Q8 Student Answers:

> __a)__ According to Lin and Dyer's consideration, from scalability perspective, an `ideal algorithm` should have 2 characteristics: 
1. If we increase the data by twice the normal size, the algorithm should take no more than twice the amount to execute the code provided everything is kept the same.
2. If we increase the computation(clusters) power by twice, the algorithm should execute the code in no more then half the time provided everything else is kept the same.

> __b)__ From below execution for $8.b$, we see that when we increase the number of resources(partitions) from 1 to 2 to 4, the corresponding time taken to execute the code is **nearly halved**. This does not meet Lin and Dyer's criteria for an `ideal algorithm` which asserts the time executed should be **at most half the time** if we increase the computation by a factor of $2$

> __c)__ From below execution for $8.c$, as the number of resources(partitions) increase, the ime taken for the entire execution of code is increased. This **does not match** our expectation and is completely against Lin and Dye's criteria for `ideal algorithm`. As its performance matches our expectation for mapper code and does not match for mapper & reducer, we can speculate that the time delays are caused by **reducer code**. So, based on our obseravtion for mapper & reducer code, we can conclude that `pWordCount_v2.sh` is not scalable implementation in terms of the resources(partitions)

> __d)__ Mapper and Reducer components of `pWordCount_v2.sh` are affected by the increase/decrease of the number of resources. Increase in the number of resources **decrease the mapper** execution time and **increase the reducer** execution time. Increase in the reducer execution time is due to **sort merging** the large number of cunks of data as the partitions increased. 


__Run the following cells to create the mapper referenced in `part b`__

In [34]:
!mkdir demo

mkdir: cannot create directory 'demo': File exists


In [35]:
%%writefile demo/mapper.py
#!/usr/bin/env python
"""
This mapper reads from STDIN and waits 0.001 seconds per line.
Its only purpose is to demonstrate one of the scalability ideas.
"""
import sys
import time
for line in sys.stdin:
    time.sleep(0.001)

Overwriting demo/mapper.py


In [36]:
# Make sure the mapper is executable
!chmod a+x demo/mapper.py

__Run the next three cells to apply our demo mapper with 1, 2 and 4 partitions.__

In [37]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'demo/mapper.py'

1 loop, best of 3: 4.79 s per loop


In [38]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'demo/mapper.py'

1 loop, best of 3: 2.75 s per loop


In [39]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'demo/mapper.py'

1 loop, best of 3: 1.78 s per loop


__Run the following cells to repeat this process with your word count algorithm.__

In [40]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

1 loop, best of 3: 790 ms per loop


In [41]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

1 loop, best of 3: 800 ms per loop


In [42]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

1 loop, best of 3: 875 ms per loop


In [43]:
%%timeit
!./pWordCount_v2.sh 8 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

1 loop, best of 3: 1.01 s per loop


In [44]:
%%timeit
!./pWordCount_v2.sh 16 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

1 loop, best of 3: 1.28 s per loop


In [45]:
%%timeit
!./pWordCount_v2.sh 32 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

1 loop, best of 3: 1.83 s per loop


# Question 9: Embarrassingly Parallel
`"If any one of them can explain it," said Alice, (she had grown so large in the last few minutes that she wasn’t a bit afraid of interrupting him,) "I’ll give him sixpence. I don’t believe there’s an atom of meaning in it."`
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 12  </div>

### Q9 Tasks:

* __a) short response:__ Describe what we mean by 'Embarrassingly Parallel' in terms of word counting. Does this term describe a 'task'? an 'implementation of a task'? 

* __b) OPTIONAL__: Define this concept in terms of 'associative' and 'commutative' operations. [`HINT:` _Refer to Week 2 Async, particularly section 2.4 and the assigned_ [reading from EECS](https://patterns.eecs.berkeley.edu/?page_id=37) _about Map-Reduce Patterns_]

### Q9 Student Answers:

> __a)__ In word counting problem, parallel processes can perform the counting of the words parllelly and reduce at the end and requires no furthur communication in between the processes. This is an example of **`perfectly parallel`** solution as the instructions are given at the beginning of the task and results are collected at the end with no process dependency. Embarrassingly Parallel solution in terms of word counting **is an implementation of a task**

> __b)__ Type your (OPTIONAL) answer here!

### Congratulations, you have completed HW1! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform